In [ ]:
import os
from pathlib import Path
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.transform import from_origin
import geopandas as gpd
import pandas as pd
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from constants import HANSEN_FILEPATHS, PROPERTIES_SHAPEFILE_PATHS, PROJECT_PATH, SHAPEFILE_PATH

# Mosaic the Hansen tiles.

In [ ]:
# Mosaic the Hansen tiles
'''def mosaic_hansen_tiles(filepaths):
    src_files_to_mosaic = [rasterio.open(fp) for fp in filepaths]
    mosaic, out_transform = merge(src_files_to_mosaic)
    out_meta = src_files_to_mosaic[0].meta.copy()
    for src in src_files_to_mosaic:
        src.close()
    return mosaic, out_transform, out_meta

mosaic, out_transform, out_meta = mosaic_hansen_tiles(HANSEN_FILEPATHS)'''

# Load the Study boundary dataset and crop the mosaic to the property boundaries.

In [ ]:
study_boundary_gdf = gpd.read_file(SHAPEFILE_PATH)


# Reproject the study boundary to match the Hansen raster CRS
with rasterio.open(HANSEN_FILEPATHS[0]) as src:
    hansen_crs = src.crs

# Reproject the study boundary GeoDataFrame to match the Hansen raster CRS    
study_boundary_gdf_reprojected = study_boundary_gdf.to_crs(hansen_crs)
# Get the geometry column from the reprojected GeoDataFrame
study_boundary_geom_reprojected = study_boundary_gdf_reprojected.geometry


In [ ]:
hansen_filepath = "/Users/romero61/../../capstone/pyforest/ml_data/hansen/clipped_hansen_lossyear.tiff"


In [ ]:
# Check the CRS of the Hansen raster
with rasterio.open(hansen_filepath) as src:
    hansen_crs = src.crs
print("Hansen CRS:", hansen_crs)

'''# Check the CRS of the study boundary
print("Properties CRS:", study_boundary_gdf.crs)'''

In [ ]:
#read the data from the Hansen raster file into a NumPy array,
with rasterio.open(hansen_filepath) as hansen:
    hansen_array = hansen.read(1)
    hansen_crs = hansen.crs
    out_transform = hansen.transform
    out_meta = hansen.meta

In [ ]:
# Crop the mosaic using the study boundary polygon
with rasterio.open(HANSEN_FILEPATHS[0]) as src:
    out_meta.update({
        'height': mosaic.shape[1],
        'width': mosaic.shape[2],
        'transform': out_transform
    })
    with rasterio.open('mosaic.tif', 'w', **out_meta) as dst:
        dst.write(mosaic)
    study_boundary_masked, _ = mask(rasterio.open('mosaic.tif'), study_boundary_geom_reprojected, crop=True, indexes=1)


In [ ]:
#print("Shape of study_boundary_masked:", study_boundary_masked.shape)
plt.imshow(hansen_array.astype(float), cmap='viridis')
plt.colorbar()
plt.title("Cropped Mosaic")

In [ ]:
# Extract pixels with a value of 11
pixels_value_11 = (hansen_array == 11).astype(int)

# Plot the pixels with a value of 11
plt.imshow(pixels_value_11, cmap='viridis')
plt.colorbar()
plt.show()